# Codex & MCP: Sharing for Scalability
Santa's Workshop Inventory demo deck


## Goal for this deck
- Show how MCP server sharing improves consistency and resource use
- Provide runnable cues for the Santa's Workshop Inventory server
- Walk through separate vs shared modes you can demo live


## Case 1: Separate mode (two servers)
- Two HTTP MCP servers: 127.0.0.1:6274 and 127.0.0.1:6275
- Elf A client binds to Server A; Elf B client binds to Server B
- Duplicate processes, duplicate in-memory inventory state


## Separate mode: runbook
```bash
source .venv/bin/activate
python run_http_server.py --port 6274 --path /mcp  # Server A
python run_http_server.py --port 6275 --path /mcp  # Server B
```
Client config examples:
```toml
[mcp_servers."workshop-elf-a"]
transport = "http"
url = "http://127.0.0.1:6274/mcp"

[mcp_servers."workshop-elf-b"]
transport = "http"
url = "http://127.0.0.1:6275/mcp"
```
Prompt cues:
- `Hello, I am Elf A. Use the workshop tools to check the current stock of "Toy Train".`
- `Hello, I am Elf B. Use the workshop tools to check the current stock of "Toy Train".`


## Separate mode: what you see
- Both elves read Toy Train stock: 50
- Elf A produces 10 Toy Trains; Server A now reports 60
- Elf B re-checks: still 50 (its server never saw the update)
- Outcomes: inconsistent data + wasted CPU/RAM from duplicate servers


## Case 2: Shared mode (one server)
- One HTTP MCP server at 127.0.0.1:6274 serving all clients
- Both Elves point to the same URL; inventory state is shared
- Less resource use and consistent results for every client


## Shared mode: runbook
```bash
source .venv/bin/activate
python run_http_server.py --port 6274 --path /mcp  # Single shared server
```
Client config (both Elves share):
```toml
[mcp_servers."santas-workshop-inventory"]
transport = "http"
url = "http://127.0.0.1:6274/mcp"
```
Prompt cues:
- `Elf A reporting in. Use the workshop tools to check the current stock of "Toy Train".`
- `Elf B reporting in. Use the workshop tools to check the current stock of "Toy Train".`
- `Produce 5 "Doll"s using the produce_toys tool.`
- `Produce 10 "Teddy Bear"s using the produce_toys tool.`


## Shared mode: results
- Both elves see Toy Train stock: 60 (single source of truth)
- Concurrent production hits the same server:
  - Elf A makes 5 Dolls -> stock 205
  - Elf B makes 10 Teddy Bears -> stock 110
- Outcomes: consistent data, less operational overhead, scalable pattern


## Takeaways and talking points
- Separate servers isolate context but fragment state and waste resources
- A shared MCP server keeps AI clients synchronized and cheaper to run
- Reuse this pattern for any tool-heavy workflow that multiple agents need
- Demo flow: start servers, run prompts above, call out the contrasting results


## Countdown to the big days
<div class="countdown-grid" style="display: grid; grid-template-columns: repeat(auto-fit, minmax(240px, 1fr)); gap: 18px; align-items: stretch; margin-top: 0.8rem;">
  <div style="padding: 16px; border: 2px solid #e63946; border-radius: 12px; background: linear-gradient(135deg, #fff8f2, #ffe3e3); box-shadow: 0 10px 24px rgba(0,0,0,0.12);">
    <div style="font-size: 1.1rem; font-weight: 700; color: #b22222;">🎄 Christmas</div>
    <div id="christmas-timer" style="font-size: 1.8rem; font-weight: 800; color: #1f2937; margin-top: 6px;">--</div>
    <div style="font-size: 0.95rem; color: #374151;">Countdown to Dec 25</div>
  </div>
  <div style="padding: 16px; border: 2px solid #2563eb; border-radius: 12px; background: linear-gradient(135deg, #f0f7ff, #e2f2ff); box-shadow: 0 10px 24px rgba(0,0,0,0.12);">
    <div style="font-size: 1.1rem; font-weight: 700; color: #0f172a;">🎆 New Year</div>
    <div id="newyear-timer" style="font-size: 1.8rem; font-weight: 800; color: #0f172a; margin-top: 6px;">--</div>
    <div style="font-size: 0.95rem; color: #374151;">Countdown to Jan 1</div>
  </div>
  <div style="padding: 16px; border: 2px solid #10b981; border-radius: 12px; background: linear-gradient(135deg, #f0fff7, #dcfce7); box-shadow: 0 10px 24px rgba(0,0,0,0.12); display: flex; flex-direction: column; gap: 8px;">
    <div style="display: flex; align-items: center; gap: 10px;">
      <img src="https://static1.squarespace.com/static/6776a992b4a6c54a4f83c0b6/t/6776a99db4a6c54a4f83c1e0/1692385485284/PyData_logo_ICON.png?format=500w" alt="PyData London 2026 logo" style="width: 60px; height: 60px; object-fit: contain; border-radius: 8px; background: white; padding: 4px; box-shadow: inset 0 0 0 1px rgba(0,0,0,0.05);" />
      <div style="font-size: 1.05rem; font-weight: 750; color: #065f46; line-height: 1.2;">PyData London 2026</div>
    </div>
    <div id="pydata-timer" style="font-size: 1.8rem; font-weight: 800; color: #0f172a; margin-top: 2px;">--</div>
    <div style="font-size: 0.95rem; color: #374151;">Countdown to June 5, 2026</div>
  </div>
</div>
<div style="margin-top: 0.25rem; color: #4b5563; font-size: 0.95rem;">Live timers update every second to keep the elves on schedule.</div>
<script>
(function() {
  const christmasEl = document.getElementById('christmas-timer');
  const newYearEl = document.getElementById('newyear-timer');
  const pyDataEl = document.getElementById('pydata-timer');
  function nextChristmas(now) {
    const christmas = new Date(now.getFullYear(), 11, 25, 0, 0, 0, 0);
    if (now > christmas) christmas.setFullYear(christmas.getFullYear() + 1);
    return christmas;
  }
  function nextNewYear(now) {
    return new Date(now.getFullYear() + 1, 0, 1, 0, 0, 0, 0);
  }
  function pyData2026() {
    return new Date(2026, 5, 5, 0, 0, 0, 0);
  }
  function formatDiff(target, now) {
    const diffMs = target - now;
    if (diffMs <= 0) return '🎉 It\'s here!';
    const totalSeconds = Math.floor(diffMs / 1000);
    const days = Math.floor(totalSeconds / 86400);
    const hours = Math.floor((totalSeconds % 86400) / 3600);
    const minutes = Math.floor((totalSeconds % 3600) / 60);
    const seconds = totalSeconds % 60;
    return `${days}d ${hours}h ${minutes}m ${seconds}s`;
  }
  function tick() {
    const now = new Date();
    christmasEl.textContent = formatDiff(nextChristmas(now), now);
    newYearEl.textContent = formatDiff(nextNewYear(now), now);
    pyDataEl.textContent = formatDiff(pyData2026(), now);
  }
  tick();
  setInterval(tick, 1000);
})();
</script>
